# Chapter 48: Documentation and Standards

Documentation is not ancillary to database engineering—it is infrastructure. Without explicit standards, teams accumulate technical debt through inconsistent naming, unchecked query patterns, and tribal knowledge that vanishes when engineers depart. This chapter provides production-ready templates for SQL style conventions, review procedures, and operational runbooks that transform individual expertise into organizational capability.

## 48.1 SQL Style Guide and Conventions

### 48.1.1 Formatting and Layout Standards

Consistent formatting reduces cognitive load during code reviews and prevents syntax errors.

```sql
-- GOOD: Left-aligned keywords, indented clauses, trailing commas
SELECT 
    u.user_id,
    u.email,
    u.created_at,
    COUNT(o.order_id) AS order_count,
    SUM(o.total_cents) / 100.0 AS total_revenue
FROM 
    users AS u
    LEFT JOIN orders AS o 
        ON u.user_id = o.user_id 
        AND o.status = 'completed'
WHERE 
    u.created_at >= '2024-01-01'
    AND u.status != 'deleted'
GROUP BY 
    u.user_id,
    u.email,
    u.created_at
HAVING 
    COUNT(o.order_id) > 0
ORDER BY 
    total_revenue DESC
LIMIT 25;

-- BAD: Inconsistent indentation, right-aligned keywords, leading commas
SELECT u.user_id,
       u.email
  , u.created_at
       ,COUNT(o.order_id) order_count
  FROM users u LEFT JOIN orders o ON u.user_id=o.user_id
 WHERE u.created_at>='2024-01-01'
   AND u.status!='deleted'
 GROUP BY u.user_id,u.email,u.created_at;
```

**Formatting Rules:**

| Element | Standard | Rationale |
|---------|----------|-----------|
| **Keywords** | UPPERCASE (`SELECT`, `WHERE`) | Distinguishes SQL verbs from identifiers |
| **Identifiers** | lowercase (`user_id`, `created_at`) | POSIX compatibility, readability |
| **Indentation** | 4 spaces per level | Clear hierarchy without tab-width ambiguity |
| **Line breaks** | One clause per line | Easier git diffs, readable on narrow screens |
| **Commas** | Trailing after each column | Adding/removing last item doesn't require editing previous line |
| **Aliases** | Explicit `AS` keyword | Prevents ambiguity with reserved words |
| **Joins** | Table on same line, `ON` condition indented | Visual separation of join logic |

### 48.1.2 Naming Conventions

Consistent naming enables automated tooling and prevents ambiguity.

```sql
-- Tables: plural, snake_case, domain-prefixed if microservices
CREATE TABLE users (...);                    -- Good
CREATE TABLE user_account (...);             -- Good (compound clarity)
CREATE TABLE User (...);                     -- Bad (reserved word, singular)

-- Columns: singular, descriptive, units in name
CREATE TABLE products (
    product_id UUID PRIMARY KEY,            -- Good: type + entity + id
    name VARCHAR(255) NOT NULL,             -- Good: simple within context
    price_cents INTEGER NOT NULL,           -- Good: unit explicit (avoids float)
    price INTEGER NOT NULL,                 -- Bad: unit ambiguous (dollars? cents?)
    description TEXT,                       -- Good
    desc TEXT                               -- Bad: abbreviated, reserved word
);

-- Indexes: purpose_prefix + table + columns
CREATE INDEX idx_users_email ON users(email);                    -- Good
CREATE INDEX idx_users_created_at_status ON users(created_at, status); -- Good
CREATE INDEX email_index ON users(email);                        -- Bad: generic

-- Constraints: type_prefix + table + columns
ALTER TABLE orders 
    ADD CONSTRAINT pk_orders PRIMARY KEY (order_id),              -- pk_ prefix
    ADD CONSTRAINT fk_orders_user_id 
        FOREIGN KEY (user_id) REFERENCES users(user_id),          -- fk_ prefix
    ADD CONSTRAINT uq_orders_order_number 
        UNIQUE (order_number),                                    -- uq_ prefix
    ADD CONSTRAINT chk_orders_positive_total 
        CHECK (total_cents > 0);                                  -- chk_ prefix

-- Sequences: entity + field + seq
CREATE SEQUENCE users_user_id_seq;          -- Good (matches SERIAL convention)
CREATE SEQUENCE user_id_seq;                -- Bad: ambiguous scope
```

**Naming Prefix Reference:**

| Prefix | Usage | Example |
|--------|-------|---------|
| `pk_` | Primary Key | `pk_users` |
| `fk_` | Foreign Key | `fk_orders_user_id` |
| `uq_` | Unique Constraint | `uq_users_email` |
| `chk_` | Check Constraint | `chk_products_positive_price` |
| `idx_` | Standard Index | `idx_users_status` |
| `idxu_` | Unique Index | `idxu_users_confirmation_token` |
| `trg_` | Trigger | `trg_users_audit` |
| `vw_` | View | `vw_active_users` |
| `mv_` | Materialized View | `mv_daily_revenue` |

### 48.1.3 Commenting and Documentation Standards

Self-documenting schemas reduce onboarding time and prevent misuse.

```sql
-- Table-level documentation
COMMENT ON TABLE users IS 
    'Core user accounts. Soft-deleted records have deleted_at populated. 
     Partitioned by created_at for data older than 2 years.
     See: https://wiki.company.com/data/users';

-- Column-level documentation
COMMENT ON COLUMN users.email IS 
    'Unique email address. Normalized to lowercase on insert via trigger. 
     Used for authentication and notifications.';

COMMENT ON COLUMN users.password_hash IS 
    'Argon2id hash. Never expose in API responses. 
     Rotation required annually per security policy.';

-- Constraint documentation (why, not just what)
COMMENT ON CONSTRAINT chk_users_email_format ON users IS 
    'Ensures valid email format before application-level validation. 
     Regex: RFC 5322 simplified.';

-- Function documentation
COMMENT ON FUNCTION calculate_tax(NUMERIC, TEXT) IS 
    'Calculate sales tax based on amount and region code.
     Returns amount * tax_rate for valid regions.
     Throws exception for invalid region codes.
     Author: DataTeam
     Created: 2024-01-15
     Last Modified: 2024-06-01 (added VAT support)';
```

**Documentation Requirements:**

1. **All tables** must have comments explaining business purpose and soft-delete policy
2. **All columns** containing PII must be marked with `[PII]` in comment
3. **All functions** must include parameter descriptions, return values, and error conditions
4. **Complex joins** in views must include ERD references or business logic explanations

### 48.1.4 Query Writing Standards

Prevent common performance and correctness issues through enforced patterns.

```sql
-- ALWAYS use explicit column lists (never SELECT *)
-- Bad: SELECT * FROM users WHERE id = 1;
-- Good:
SELECT user_id, email, full_name FROM users WHERE user_id = 1;

-- ALWAYS qualify column names in multi-table queries
SELECT 
    u.user_id,
    u.email,
    o.order_id
FROM users AS u
JOIN orders AS o ON u.user_id = o.user_id;  -- Good: explicit join condition

-- NEVER use functions on indexed columns in WHERE (prevents index usage)
-- Bad:
SELECT * FROM users WHERE DATE(created_at) = '2024-01-01';
-- Good:
SELECT * FROM users 
WHERE created_at >= '2024-01-01' 
  AND created_at < '2024-01-02';

-- ALWAYS handle NULL explicitly in comparisons
-- Bad:
SELECT * FROM orders WHERE cancelled_at != '2024-01-01';
-- Good:
SELECT * FROM orders 
WHERE cancelled_at IS DISTINCT FROM '2024-01-01'::date;

-- NEVER use IN with large lists (performance degrades >100 items)
-- Bad:
SELECT * FROM users WHERE user_id IN (1, 2, 3, ... 1000);
-- Good:
CREATE TEMP TABLE tmp_user_ids (id INT PRIMARY KEY);
INSERT INTO tmp_user_ids VALUES (1), (2), ...;
SELECT u.* FROM users u
JOIN tmp_user_ids t ON u.user_id = t.id;
```

## 48.2 Schema Review Checklist

### 48.2.1 Pre-Commit Checklist (Developer Self-Review)

Before submitting a migration for peer review, verify:

```markdown
## Schema Change Self-Review Checklist

### Correctness
- [ ] All tables have primary keys (surrogate UUID or BIGINT, never natural keys alone)
- [ ] Foreign keys have matching data types and collations
- [ ] Check constraints use immutable functions only
- [ ] Default values match column types (e.g., `NOW()` for `TIMESTAMPTZ`, not `TIMESTAMP`)
- [ ] Enum values are comprehensive (include 'unknown' or 'other' if applicable)

### Performance
- [ ] Indexes added for all foreign key columns (unless table < 1000 rows)
- [ ] Composite indexes follow column ordering rules (equality first, range second)
- [ ] Partial indexes use `WHERE` conditions matching common queries
- [ ] Large text columns use `TEXT` not `VARCHAR(n)` (no performance difference, better compatibility)
- [ ] JSONB preferred over JSON for indexing and querying

### Safety
- [ ] New columns added as nullable OR with defaults (never `NOT NULL` without default on existing tables)
- [ ] Dangerous operations (DROP COLUMN, RENAME) use Expand-Contract pattern documented
- [ ] `ON DELETE` specified for all foreign keys (default is RESTRICT, consider CASCADE carefully)
- [ ] Column removal preceded by application code deprecation (dual-write period confirmed)

### Security
- [ ] No plaintext passwords or secrets in schema
- [ ] PII columns marked with comments
- [ ] Row Level Security (RLS) policies added if table contains user data
- [ ] Default privileges reviewed for new schemas

### Maintainability
- [ ] Consistent naming conventions followed (see Style Guide 48.1)
- [ ] Comments added for all objects
- [ ] Migration is idempotent (can run multiple times without error)
- [ ] Down migration tested (if using reversible migration tool)
```

### 48.2.2 Peer Review Checklist (Senior Engineer/DBA)

Deep review for production safety:

```markdown
## Production Schema Review Checklist

### Data Integrity
- [ ] **Cardinality Check**: Foreign key relationships match business logic (1:1, 1:N, M:N correctly modeled)
- [ ] **Temporal Integrity**: Timezone handling consistent (`TIMESTAMPTZ` vs `TIMESTAMP`), no mixing
- [ ] **Constraint Validation**: Check constraints handle edge cases (empty strings, zero values, extreme dates)
- [ ] **Uniqueness Scope**: Unique constraints account for soft deletes (use partial indexes `WHERE deleted_at IS NULL`)

### Locking and Downtime
- [ ] **Lock Duration**: `ALTER TABLE` operations estimated for table size (<1s for <1M rows, <5s for <10M rows)
- [ ] **Concurrent Index Creation**: New indexes use `CREATE INDEX CONCURRENTLY` (not blocking)
- [ ] **Constraint Validation**: Foreign keys added as `NOT VALID` then `VALIDATE CONSTRAINT` in separate transaction
- [ ] **Table Rewrite Avoidance**: Verify `ALTER` doesn't force table rewrite (check `pg_catalog` for "requires rewrite")

### Scalability
- [ ] **Partitioning Strategy**: Tables >100GB or >100M rows have partitioning plan
- [ ] **TOAST Evaluation**: Large columns (>2KB) understood and acceptable
- [ ] **Autovacuum Tuning**: High-churn tables have custom autovacuum settings
- [ ] **Connection Impact**: New sequences have appropriate cache size for high-insert scenarios

### Observability
- [ ] **Monitoring Alerts**: New tables added to "table bloat" monitoring dashboard
- [ ] **SLA Alignment**: Slow query thresholds defined for new access patterns
- [ ] **Dependency Documentation**: Downstream ETL/data warehouse impact assessed
```

### 48.2.3 Migration Safety Verification Script

```sql
-- Run this before approving schema changes to production
-- Save as: scripts/schema_safety_check.sql

DO $$
DECLARE
    r RECORD;
    issues TEXT[] := ARRAY[]::TEXT[];
BEGIN
    -- Check 1: Unindexed foreign keys
    FOR r IN 
        SELECT
            tc.table_name,
            kcu.column_name,
            ccu.table_name AS foreign_table_name
        FROM 
            information_schema.table_constraints AS tc
            JOIN information_schema.key_column_usage AS kcu
              ON tc.constraint_name = kcu.constraint_name
            JOIN information_schema.constraint_column_usage AS ccu
              ON ccu.constraint_name = tc.constraint_name
        WHERE tc.constraint_type = 'FOREIGN KEY'
          AND NOT EXISTS (
              SELECT 1 FROM pg_indexes pi
              WHERE pi.tablename = tc.table_name
                AND pi.indexdef LIKE '%' || kcu.column_name || '%'
          )
    LOOP
        issues := array_append(issues, 
            format('Unindexed FK: %s(%s) -> %s', 
                   r.table_name, r.column_name, r.foreign_table_name));
    END LOOP;
    
    -- Check 2: Missing primary keys
    FOR r IN 
        SELECT tablename 
        FROM pg_tables 
        WHERE schemaname = 'public'
          AND NOT EXISTS (
              SELECT 1 FROM pg_constraint 
              WHERE conrelid = (schemaname || '.' || tablename)::regclass
                AND contype = 'p'
          )
    LOOP
        issues := array_append(issues, format('Missing PK: %s', r.tablename));
    END LOOP;
    
    -- Check 3: Tables without comments
    FOR r IN 
        SELECT c.relname 
        FROM pg_class c
        JOIN pg_namespace n ON n.oid = c.relnamespace
        WHERE c.relkind = 'r' 
          AND n.nspname = 'public'
          AND obj_description(c.oid) IS NULL
    LOOP
        issues := array_append(issues, format('Missing comment: %s', r.relname));
    END LOOP;
    
    -- Report
    IF array_length(issues, 1) > 0 THEN
        RAISE WARNING 'Schema issues found: %', issues;
    ELSE
        RAISE NOTICE 'All schema checks passed';
    END IF;
END $$;
```

## 48.3 Query Review Checklist

### 48.3.1 Performance Review Criteria

Every query entering production code must pass these checks:

```markdown
## Query Production Readiness Review

### Execution Plan Analysis
- [ ] **EXPLAIN ANALYZE** executed on production-size dataset (not just dev)
- [ ] **No Seq Scans** on tables >10,000 rows (unless intentionally full-table scan)
- [ ] **Index Usage**: Bitmap scans acceptable, but verify bitmap heap ratio
- [ ] **Join Order**: Smallest result set joined first (check with `SET join_collapse_limit = 1` if uncertain)
- [ ] **Sort Operations**: In-memory sorts only (`Sort Method: quicksort`), no disk sorts
- [ ] **Parallelism**: Queries >100ms utilize parallel workers if beneficial

### Resource Consumption
- [ ] **Execution Time**: <50ms for OLTP (user-facing), <5s for reporting
- [ ] **Memory Usage**: Work_mem sufficient (no `Disk Spool` operations)
- [ ] **Locking**: No `FOR UPDATE` on large ranges without `LIMIT`
- [ ] **Write Impact**: `UPDATE`/`DELETE` limited to <1000 rows per transaction (batch if larger)

### Correctness
- [ ] **NULL Handling**: Three-valued logic considered (`NULL != NULL` evaluated correctly)
- [ ] **Duplicate Check**: Joins don't produce Cartesian products (verify row counts)
- [ ] **Time Zone**: `TIMESTAMPTZ` used consistently, no mixing with `TIMESTAMP`
- [ ] **Collation**: Text comparisons specify collation if locale-dependent sorting required

### Maintainability
- [ ] **Parameterization**: All user inputs parameterized (prevents SQL injection)
- [ ] **CTE Usage**: CTEs marked `MATERIALIZED` or `NOT MATERIALIZED` explicitly (don't rely on defaults)
- [ ] **Pagination**: Offset/limit only for small tables (<10k rows), keyset pagination for large tables
```

### 48.3.2 Anti-Pattern Detection

Automated checks for common mistakes:

```sql
-- Check for SELECT * in views (brittle to schema changes)
SELECT viewname, definition 
FROM pg_views 
WHERE definition ILIKE '%SELECT *%'
  AND schemaname = 'public';

-- Check for implicit cross joins (missing join condition)
-- Review query plans for "Nested Loop" with high row counts
-- Look for: Join Filter vs actual Join Condition

-- Check for functions on indexed columns (performance killers)
-- Review slow query logs for patterns like:
-- WHERE upper(email) = 'USER@EXAMPLE.COM'  -- Should use functional index
-- WHERE created_at::date = '2024-01-01'    -- Should use range

-- Check for missing LIMIT on user-facing queries
-- Any SELECT without LIMIT in application code paths should be flagged
```

### 48.3.3 Query Review Template

```markdown
## Query Review Request

**Query Purpose**: [Brief description of business logic]

**Data Volume**:
- Primary table row count: [e.g., users: 2M rows]
- Expected result set size: [e.g., 50 rows per query]

**Execution Plan**:
```sql
-- Paste EXPLAIN (ANALYZE, BUFFERS, FORMAT JSON) output here
```

**Indexes Required**:
- [ ] Existing index sufficient: `idx_users_email`
- [ ] New index required: `CREATE INDEX CONCURRENTLY idx_orders_user_status ON orders(user_id, status)`

**Risk Assessment**:
- [ ] Read-only query (safe)
- [ ] Write query (requires transaction boundary review)
- [ ] Concurrent with high write volume (risk of lock contention)

**Rollback Plan**:
- [ ] Can be killed without side effects
- [ ] Wrapped in retry logic with exponential backoff
- [ ] Circuit breaker implemented at application layer
```

## 48.4 Operational Runbook Templates

### 48.4.1 Incident Response Runbook Template

Standardized response to database incidents:

```markdown
# Database Incident Response Runbook

## Severity Levels

| Level | Criteria | Response Time | Escalation |
|-------|----------|---------------|------------|
| SEV1 | Complete outage, data loss, corruption | 5 minutes | Immediate CEO/CTO |
| SEV2 | Degraded performance (>5s latency) | 15 minutes | Engineering Manager |
| SEV3 | Single replica failure, non-critical alerts | 1 hour | On-call DBA |

## Initial Response (First 5 Minutes)

1. **Assess Scope**
   ```sql
   -- Check current connections and activity
   SELECT count(*), state FROM pg_stat_activity GROUP BY state;
   
   -- Check replication lag
   SELECT extract(epoch from (now() - pg_last_xact_replay_timestamp())) AS lag_seconds;
   
   -- Check locks
   SELECT * FROM pg_locks WHERE NOT granted;
   ```

2. **Communication**
   - Post in #incidents channel: "Investigating database [symptom]"
   - Create incident bridge call if SEV1/SEV2

3. **Safety First**
   - Do NOT restart primary without understanding root cause
   - Do NOT kill queries indiscriminately (may be replication processes)

## Common Scenarios

### Scenario A: Lock Contention (Queries hanging)

**Diagnosis**:
```sql
-- Find blocking queries
SELECT blocked_locks.pid AS blocked_pid,
       blocked_activity.usename AS blocked_user,
       blocking_locks.pid AS blocking_pid,
       blocking_activity.query AS blocking_statement
FROM pg_catalog.pg_locks blocked_locks
JOIN pg_catalog.pg_stat_activity blocked_activity ON blocked_activity.pid = blocked_locks.pid
JOIN pg_catalog.pg_locks blocking_locks 
    ON blocking_locks.locktype = blocked_locks.locktype
    AND blocking_locks.relation = blocked_locks.relation
    AND blocking_locks.pid != blocked_locks.pid
JOIN pg_catalog.pg_stat_activity blocking_activity ON blocking_activity.pid = blocking_locks.pid
WHERE NOT blocked_locks.granted;
```

**Resolution**:
1. Identify if blocking query is legitimate (migration, batch job)
2. If illegitimate: `SELECT pg_terminate_backend(blocking_pid);`
3. If legitimate but stuck: Evaluate if safe to cancel or must wait

### Scenario B: Disk Space Critical

**Diagnosis**:
```bash
df -h /var/lib/postgresql/data
du -sh /var/lib/postgresql/data/pg_wal/* | sort -h | tail -10
```

**Immediate Actions**:
1. Check replication slots: `SELECT * FROM pg_replication_slots;` (inactive slots cause WAL accumulation)
2. If safe, drop inactive slots: `SELECT pg_drop_replication_slot('slot_name');`
3. Truncate logs if separate volume: Do NOT touch pg_wal directly

### Scenario C: Connection Exhaustion

**Immediate**:
```sql
-- Increase max_connections temporarily (requires restart, so plan B):
-- Instead, terminate idle connections:
SELECT pg_terminate_backend(pid) 
FROM pg_stat_activity 
WHERE state = 'idle' 
  AND state_change < now() - interval '5 minutes';
```

## Post-Incident Review

Required within 24 hours:
- [ ] Timeline of events documented
- [ ] Root cause identified (5 Whys)
- [ ] Prevention measures added to runbook
- [ ] Monitoring gaps identified and tickets created
```

### 48.4.2 Maintenance Runbook Template

Scheduled maintenance procedures:

```markdown
# Monthly Database Maintenance

## Pre-Maintenance Checklist (24 hours prior)

- [ ] Confirm maintenance window with stakeholders
- [ ] Verify backups: `pg_dump` and PITR both validated
- [ ] Check replication lag < 1 second on all replicas
- [ ] Prepare rollback scripts for each change
- [ ] Notify on-call engineer of maintenance window

## Maintenance Procedures

### 1. Index Maintenance (Bloat Removal)

**Safety Check**:
```sql
-- Estimate bloat
SELECT schemaname, tablename, 
       pg_size_pretty(pg_total_relation_size(schemaname||'.'||tablename)) as size,
       n_dead_tup, n_live_tup,
       round(n_dead_tup::numeric/nullif(n_live_tup,0)*100, 2) as dead_pct
FROM pg_stat_user_tables 
WHERE n_dead_tup > 10000
ORDER BY n_dead_tup DESC;
```

**Procedure**:
```bash
# Rebuild bloated indexes concurrently (no locking)
psql -c "REINDEX INDEX CONCURRENTLY idx_users_email;"
psql -c "REINDEX INDEX CONCURRENTLY idx_orders_created_at;"
```

### 2. Statistics Update

```sql
-- Update all table statistics (during low traffic)
ANALYZE VERBOSE;

-- For specific large tables
ANALYZE orders;
ANALYZE user_events;
```

### 3. Log Rotation and Archival

```bash
# Rotate PostgreSQL logs
pg_ctl rotate

# Archive logs older than 30 days to S3
aws s3 sync /var/log/postgresql/ s3://backups/postgres-logs/ --exclude "*" --include "*.gz"
find /var/log/postgresql/ -name "*.log" -mtime +7 -delete
```

## Post-Maintenance Validation

- [ ] Application health checks pass
- [ ] Replication lag normal (< 5 seconds)
- [ ] No new errors in logs
- [ ] Performance metrics baseline compared to pre-maintenance
```

### 48.4.3 Access Management Runbook

```markdown
# Database Access Provisioning

## Onboarding New Developer

1. **Principle of Least Privilege**
   - Development: Read/write on dev environment only
   - Staging: Read-only access
   - Production: No direct access (use read replica or audit tool)

2. **User Creation**:
```sql
-- Development access
CREATE ROLE dev_username WITH LOGIN PASSWORD 'temporary_password';
GRANT CONNECT ON DATABASE app_development TO dev_username;
GRANT USAGE ON SCHEMA public TO dev_username;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA public TO dev_username;
ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO dev_username;
```

3. **Security Requirements**:
- [ ] Password rotated within 24 hours (not default)
- [ ] SSL connection enforced
- [ ] IP allowlist configured if applicable
- [ ] Access logged in access control spreadsheet

## Offboarding

```sql
-- Immediate revocation
REVOKE ALL PRIVILEGES ON ALL TABLES IN SCHEMA public FROM leaver_username;
REVOKE ALL PRIVILEGES ON ALL SEQUENCES IN SCHEMA public FROM leaver_username;
REVOKE ALL PRIVILEGES ON ALL FUNCTIONS IN SCHEMA public FROM leaver_username;
DROP ROLE IF EXISTS leaver_username;
```

## Emergency Access (Break Glass)

1. **Request**: Slack #dba-requests with business justification
2. **Approval**: Second senior engineer approval required
3. **Provisioning**: Time-boxed (max 4 hours), specific tables only
4. **Audit**: All queries logged to separate audit table
5. **Revocation**: Automatic at expiration, manual confirmation within 1 hour
```

---

## Chapter Summary

In this chapter, you learned:

1. **SQL Style Standards**: Enforce UPPERCASE keywords, lowercase identifiers with explicit `AS` aliases, trailing commas, and 4-space indentation; document all objects with comments explaining business purpose and PII classification; use consistent prefixes (`pk_`, `fk_`, `idx_`) for constraints and indexes to enable automated tooling.

2. **Schema Review Discipline**: Implement dual-phase reviews—developer self-review (checking idempotency, NOT NULL safety, and index coverage) and senior/DBA review (verifying locking behavior, partition strategies, and backward compatibility); use automated safety scripts to detect unindexed foreign keys and missing primary keys before production deployment.

3. **Query Review Procedures**: Require `EXPLAIN ANALYZE` evidence for all production queries, enforce <50ms latency for OLTP workloads, prohibit sequential scans on large tables, and mandate keyset pagination for large result sets; maintain review templates that document execution plans, required indexes, and rollback strategies.

4. **Operational Runbooks**: Maintain living documents for incident response (including severity classifications, lock contention resolution, and disk space emergencies), scheduled maintenance (index bloat removal, statistics updates, log rotation), and access management (onboarding/offboarding procedures, break-glass protocols); ensure all runbooks include copy-paste SQL and commands tested against production-like environments.

---

This completes the **PostgreSQL Developer Handbook**. You now possess comprehensive knowledge spanning local development workflows, testing strategies, CI/CD automation, and operational standards required to build, deploy, and maintain production PostgreSQL systems at scale.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='47. cicd_for_databases.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <span style='color:gray; font-size:1.05em;'>Next</span>
</div>
